In [0]:
#import all the necessary packages.
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from bs4 import BeautifulSoup

import math
import time
import re
import os
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout

plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")

import gensim
import random
import pickle
from gensim.models import Word2Vec, KeyedVectors
# from sklearn.metrics.pairwise import cosine_similarity
# import nltk
# from nltk.corpus import stopwords
from nltk.corpus import stopwords
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
import nltk
nltk.download('stopwords')
!pip install distance
import distance

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 184kB 2.7MB/s 
  Created wheel for distance: filename=Distance-0.1.3-cp36-none-any.whl size=16261 sha256=318e22b325305655c2c3d5df498465e14bda67606c0d589c4b3746cf0a875737
  Stored in directory: /root/.cache/pip/wheels/d5/aa/e1/dbba9e7b6d397d645d0f12db1c66dbae9c5442b39b001db18e
Successfully built distance


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/Train.csv')
# data = pd.read_csv('Train.csv')
print(data.shape)
data.head()

(31499, 3)


,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [0]:
data.distractor = [i.split(',') for i in data.distractor]
data = data.explode('distractor')
#converting to lower case
data = data.apply(lambda x: x.astype(str).str.lower())
import re
# replacing _' with '
data = data.apply(lambda x: x.astype(str).str.replace(r" '", r"'"))
# replacing ( n't) with (n't)
data = data.apply(lambda x: x.astype(str).str.replace(r" n't", r"n't"))
# replacing ' s with 's
data = data.apply(lambda x: x.astype(str).str.replace(r" 's", r"'s"))
# removing speacial charecters
data = data.apply(lambda x: x.astype(str).str.replace(r"[^A-Za-z0-9\s]+", r""))
data.head()

,question,answer_text,distractor
0,meals can be served,in rooms at 900 p m,outside the room at 300 p m
0,meals can be served,in rooms at 900 p m,in the dining room at 600 p m
0,meals can be served,in rooms at 900 p m,in the dining room from 730 a m to 915 p m
1,it can be inferred from the passage that,the local government can deal with the problem...,if some tragedies occur again
1,it can be inferred from the passage that,the local government can deal with the problem...,relevant departments of the state council sho...


In [0]:
# Collecting all the question and option strings to create a seperate w2v model for only questions
questions = data.question.drop_duplicates()
answer_strings = data.answer_text.drop_duplicates()
distractor_strings = data.distractor.drop_duplicates()
options = pd.concat([answer_strings, distractor_strings])
questions = questions.apply(lambda x: x.split(" "))
options = options.apply(lambda x: x.split(" "))
options.head()

0                           [in, rooms, at, 900, p, m]
1    [the, local, government, can, deal, with, the,...
2    [help, them, realize, their, influence, on, to...
3    [the, writer, is, not, very, willing, to, use,...
4                      [stay, calm, and, dont, move, ]
dtype: object

In [0]:
X_questions = questions.to_numpy()
X_options = options.to_numpy()

In [0]:
X_options

array([list(['in', 'rooms', 'at', '900', 'p', 'm']),
       list(['the', 'local', 'government', 'can', 'deal', 'with', 'the', 'problem', 'of', 'lacking', 'money', 'by', 'some', 'means', '']),
       list(['help', 'them', 'realize', 'their', 'influence', 'on', 'tommy']),
       ..., list(['he', 'liked', 'go', 'play', 'with', 'herbs', '']),
       list(['he', 'studied', 'at', 'a', 'herbal', 'medicine', 'store']),
       list(['his', 'family', 'had', 'a', 'herbal', 'medicine', 'store'])],
      dtype=object)

In [0]:
%%time
w2v_ques_model = Word2Vec(X_questions, size=100, workers=4)
w2v_ques_model.save('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/ques_word2vec.model')
# w2v_ques_model.save('ques_word2vec.model')

CPU times: user 2.68 s, sys: 22.2 ms, total: 2.71 s
Wall time: 2.49 s


In [0]:
for index, i in enumerate(X_options):
    X_options[index] = list(filter(lambda a: a != '', i))


In [0]:
# X_options = np.unique(X_options)
w2v_opt_model = Word2Vec(X_options, size=100, workers=4)
# w2v_opt_model.save('opt_word2vec.model')
w2v_opt_model.save('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/opt_word2vec.model')

In [0]:
def get_word_vec(sentence, text_type):
    if text_type == 'question':
        model = w2v_ques_model
        vocab = list(w2v_ques_model.wv.vocab)
    else:
        model = w2v_opt_model
        vocab = list(w2v_opt_model.wv.vocab)

    vec = []
    for i in sentence:
        if i in vocab:
            vec.append(model[i])
        else:
            vec.append(np.ones(shape=(100,)))
    if len(sentence)==0:
        vec.append(np.ones(shape=(100,)))
    return np.mean(vec, axis=0)

In [0]:
%%time
change = False
if change or not os.path.isfile('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/X_ques_vec.pkl'):
    X_ques_vec = {' '.join(xi):get_word_vec(xi, 'question') for xi in X_questions}
    f = open("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/X_ques_vec.pkl","wb")
    pickle.dump(X_ques_vec,f)
    f.close()
if change or not os.path.isfile('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/X_opt_vec.pkl'):
    X_opt_vec = {' '.join(xi):get_word_vec(xi, 'option') for xi in X_options}
    f = open("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/X_opt_vec.pkl","wb")
    pickle.dump(X_opt_vec,f)
    f.close()

f = open("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/X_ques_vec.pkl",'rb')
X_ques_vec = pickle.load(f)
f.close()

f = open("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/X_opt_vec.pkl",'rb')
X_opt_vec = pickle.load(f)
f.close()

CPU times: user 456 ms, sys: 117 ms, total: 573 ms
Wall time: 4.31 s


In [0]:
# len(X_opt_vec)
X_options.shape

(99388,)

In [0]:
## Creating new dataset with negative values for non generators
# %%time
change = False
if change or not os.path.isfile('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3.csv'):
    data2 = pd.DataFrame(columns=data.columns)
    for index, i in enumerate(data.values):
        question = i[0]
        ans = i[1]
        distractor = i[2]
        l = len(X_options)
        new_dis = ' '.join(X_options[random.randint(0, l)])
        data2 = data2.append(pd.Series([question, ans, new_dis], index=data2.columns ), ignore_index=True)
    data['target'] = 1
    data2['target'] = 0
    data3 = pd.concat([data,data2], axis=0)
    
    data3.to_csv('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3.csv')

data3 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3.csv')

In [0]:
# dropping unnecessary columns
data3.drop(data3.columns[data3.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
# Dropping values with na
data3 = data3.dropna()

In [0]:
def normalized_word_Common_qd(row):
    wq = set(map(lambda word: word.lower().strip(), row['question'].split(" ")))
    wd = set(map(lambda word: word.lower().strip(), row['distractor'].split(" ")))    
    return 1.0 * len(wq & wd)

def normalized_word_Common_ad(row):
    wa = set(map(lambda word: word.lower().strip(), row['answer_text'].split(" ")))    
    wd = set(map(lambda word: word.lower().strip(), row['distractor'].split(" ")))    
    return 1.0 * len(wa & wd)

def normalized_word_Total_qd(row):
    wq = set(map(lambda word: word.lower().strip(), row['question'].split(" ")))
    wd = set(map(lambda word: word.lower().strip(), row['distractor'].split(" ")))    
    return 1.0 * (len(wq) + len(wd))
    
def normalized_word_Total_ad(row):
    wa = set(map(lambda word: word.lower().strip(), row['answer_text'].split(" ")))
    wd = set(map(lambda word: word.lower().strip(), row['distractor'].split(" ")))    
    return 1.0 * (len(wa) + len(wd))

def normalized_word_share_qd(row):
    wq = set(map(lambda word: word.lower().strip(), row['question'].split(" ")))
    wd = set(map(lambda word: word.lower().strip(), row['distractor'].split(" ")))    
    return 1.0 * len(wq & wd)/(len(wq) + len(wd))

def normalized_word_share_ad(row):
    wa = set(map(lambda word: word.lower().strip(), row['answer_text'].split(" ")))
    wd = set(map(lambda word: word.lower().strip(), row['distractor'].split(" ")))    
    return 1.0 * len(wa & wd)/(len(wa) + len(wd))


In [0]:
%%time


if not os.path.isfile('/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_BasicFeature.csv'):
    # string length of question, answer_text, distractor
    data3['qlen'] = data3['question'].str.len() 
    data3['alen'] = data3['answer_text'].str.len() 
    data3['dlen'] = data3['distractor'].str.len() 

    # length of sentance of question, answer, distractor
    data3['q_n_words'] = data3['question'].apply(lambda row: len(row.split(" ")))
    data3['a_n_words'] = data3['answer_text'].apply(lambda row: len(row.split(" ")))
    data3['d_n_words'] = data3['distractor'].apply(lambda row: len(row.split(" ")))

    # common word count of question and distractor
    
    data3['word_Common_qd'] = data3.apply(normalized_word_Common_qd, axis=1)

    # common word count of answer_text and distractor
    
    data3['word_Common_ad'] = data3.apply(normalized_word_Common_ad, axis=1)

    # Total word count in question and distractor
    
    data3['word_Total_qd'] = data3.apply(normalized_word_Total_qd, axis=1)

    # Total word count in answer_text and distractor
    
    data3['word_Total_ad'] = data3.apply(normalized_word_Total_ad, axis=1)


    # word share of question and distractor: word share = (word_common)/(word_total)
    
    data3['word_share_qd'] = data3.apply(normalized_word_share_qd, axis=1)

    # word share of answer_text and distractor: word share = (word_common)/(word_total)

    data3['word_share_ad'] = data3.apply(normalized_word_share_ad, axis=1)

    data3.to_csv("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_BasicFeature.csv", index=False)

data3 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_BasicFeature.csv")

CPU times: user 378 ms, sys: 33.1 ms, total: 411 ms
Wall time: 1.62 s


In [0]:


SAFE_DIV = 0.0001 

STOP_WORDS = stopwords.words("english")

def get_token_features(q1, q2):
    token_features = [0.0]*10
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features
    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2
    return token_features

# get the Longest Common sub string

def get_longest_substr_ratio(a, b):
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)


In [0]:
%%time
if not os.path.isfile("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_with_nlp.csv"):
    print("token features...")

    # Merging Features with dataset - Token features for question and distractor 
    token_features = data3.apply(lambda x: get_token_features(x["question"], x["distractor"]), axis=1)

    data3["cwc_min_qd"]       = list(map(lambda x: x[0], token_features))
    data3["cwc_max_qd"]       = list(map(lambda x: x[1], token_features))
    data3["csc_min_qd"]       = list(map(lambda x: x[2], token_features))
    data3["csc_max_qd"]       = list(map(lambda x: x[3], token_features))
    data3["ctc_min_qd"]       = list(map(lambda x: x[4], token_features))
    data3["ctc_max_qd"]       = list(map(lambda x: x[5], token_features))
    data3["last_word_eq_qd"]  = list(map(lambda x: x[6], token_features))
    data3["first_word_eq_qd"] = list(map(lambda x: x[7], token_features))
    data3["abs_len_diff_qd"]  = list(map(lambda x: x[8], token_features))
    data3["mean_len_qd"]      = list(map(lambda x: x[9], token_features))

    # Merging Features with dataset - Token features for answer_text and distractor 
    token_features = data3.apply(lambda x: get_token_features(x["answer_text"], x["distractor"]), axis=1)

    data3["cwc_min_ad"]       = list(map(lambda x: x[0], token_features))
    data3["cwc_max_ad"]       = list(map(lambda x: x[1], token_features))
    data3["csc_min_ad"]       = list(map(lambda x: x[2], token_features))
    data3["csc_max_ad"]       = list(map(lambda x: x[3], token_features))
    data3["ctc_min_ad"]       = list(map(lambda x: x[4], token_features))
    data3["ctc_max_ad"]       = list(map(lambda x: x[5], token_features))
    data3["last_word_eq_ad"]  = list(map(lambda x: x[6], token_features))
    data3["first_word_eq_ad"] = list(map(lambda x: x[7], token_features))
    data3["abs_len_diff_ad"]  = list(map(lambda x: x[8], token_features))
    data3["mean_len_ad"]      = list(map(lambda x: x[9], token_features))



    print("fuzzy features..")
    data3["token_set_ratio_qd"]       = data3.apply(lambda x: fuzz.token_set_ratio(x["question"], x["distractor"]), axis=1)
    data3["token_set_ratio_ad"]       = data3.apply(lambda x: fuzz.token_set_ratio(x["answer_text"], x["distractor"]), axis=1)

    data3["token_sort_ratio_qd"]      = data3.apply(lambda x: fuzz.token_sort_ratio(x["question"], x["distractor"]), axis=1)
    data3["token_sort_ratio_ad"]      = data3.apply(lambda x: fuzz.token_sort_ratio(x["answer_text"], x["distractor"]), axis=1)

    data3["fuzz_ratio_qd"]            = data3.apply(lambda x: fuzz.QRatio(x["question"], x["distractor"]), axis=1)
    data3["fuzz_ratio_ad"]            = data3.apply(lambda x: fuzz.QRatio(x["answer_text"], x["distractor"]), axis=1)

    data3["fuzz_partial_ratio_qd"]    = data3.apply(lambda x: fuzz.partial_ratio(x["question"], x["distractor"]), axis=1)
    data3["fuzz_partial_ratio_ad"]    = data3.apply(lambda x: fuzz.partial_ratio(x["answer_text"], x["distractor"]), axis=1)

    data3["longest_substr_ratio_qd"]  = data3.apply(lambda x: get_longest_substr_ratio(x["question"], x["distractor"]), axis=1)
    data3["longest_substr_ratio_ad"]  = data3.apply(lambda x: get_longest_substr_ratio(x["answer_text"], x["distractor"]), axis=1)

    data3.to_csv("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_with_nlp.csv", index=False)

data3 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_with_nlp.csv")

CPU times: user 820 ms, sys: 66.3 ms, total: 886 ms
Wall time: 2.55 s


In [0]:
%%time
if not os.path.isfile("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_final.csv"):

    from tqdm import tqdm
    q_vec = pd.DataFrame()
    a_vec = pd.DataFrame()
    d_vec = pd.DataFrame()

    for i in tqdm(data3.values):
        q_vec = q_vec.append([X_ques_vec.get(i[0], np.zeros(100))], ignore_index=True)
        a_vec = a_vec.append( [X_opt_vec.get(i[1], np.zeros(100))], ignore_index=True)
        d_vec = d_vec.append([X_opt_vec.get(i[2], np.zeros(100))], ignore_index=True)
    data3 = pd.concat([data3, q_vec, a_vec, d_vec], axis = 1)
    print(data3.shape)
    data3.to_csv("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_final.csv", index=False)
data3 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_final.csv")

CPU times: user 9.34 s, sys: 552 ms, total: 9.89 s
Wall time: 10.8 s


In [0]:
# data3 = pd.concat([data3, q_vec, a_vec, d_vec], axis = 1)
# print(data3.shape)
# data3.to_csv("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/data3_final.csv", index=False)

In [0]:
data3.columns

# Training ML Model

In [0]:
from xgboost import XGBClassifier
clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
%%time
# X_train = data3.drop(['question', 'answer_text', 'distractor', 'target'], axis=1).to_numpy()
# y_train = data3.target
# model = clf.fit(X_train, y_train)
# # pickle.dump(model, open("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/final_model.model", "wb"))
model = pickle.load(open("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/final_model.model", "rb"))


CPU times: user 99 µs, sys: 3.92 ms, total: 4.02 ms
Wall time: 775 ms


# Applying model on Test data

In [0]:
def predict_distractor(df):
    #preprocessing
    #converting to lower case
    df = df.apply(lambda x: x.astype(str).str.lower())
    import re
    # replacing _' with '
    df = df.apply(lambda x: x.astype(str).str.replace(r" '", r"'"))
    # replacing ( n't) with (n't)
    df = df.apply(lambda x: x.astype(str).str.replace(r" n't", r"n't"))
    # replacing ' s with 's
    df = df.apply(lambda x: x.astype(str).str.replace(r" 's", r"'s"))
    # removing speacial charecters
    df = df.apply(lambda x: x.astype(str).str.replace(r"[^A-Za-z0-9\s]+", r""))

    #Applying all possible distractors
    n = X_options.shape[0]
    df = pd.concat([df]*n, ignore_index=True)
    df['distractor'] = pd.Series(X_options).apply(lambda row: ' '.join(row))
    print(df.values[100])
    # basic features
    # string length of question, answer_text, distractor
    df['qlen'] = df['question'].str.len() 
    df['alen'] = df['answer_text'].str.len() 
    df['dlen'] = df['distractor'].str.len() 
    # length of sentance of question, answer, distractor
    df['q_n_words'] = df['question'].apply(lambda row: len(row.split(" ")))
    df['a_n_words'] = df['answer_text'].apply(lambda row: len(row.split(" ")))
    df['d_n_words'] = df['distractor'].apply(lambda row: len(row))
    # common word count of question and distractor
    df['word_Common_qd'] = df.apply(normalized_word_Common_qd, axis=1)
    # common word count of answer_text and distractor
    df['word_Common_ad'] = df.apply(normalized_word_Common_ad, axis=1)
    # Total word count in question and distractor
    df['word_Total_qd'] = df.apply(normalized_word_Total_qd, axis=1)
    # Total word count in answer_text and distractor
    df['word_Total_ad'] = df.apply(normalized_word_Total_ad, axis=1)
    # word share of question and distractor: word share = (word_common)/(word_total)
    df['word_share_qd'] = df.apply(normalized_word_share_qd, axis=1)
    # word share of answer_text and distractor: word share = (word_common)/(word_total)
    df['word_share_ad'] = df.apply(normalized_word_share_ad, axis=1)

    # nlp features
    token_features = df.apply(lambda x: get_token_features(x["question"], x["distractor"]), axis=1)
    df["cwc_min_qd"]       = list(map(lambda x: x[0], token_features))
    df["cwc_max_qd"]       = list(map(lambda x: x[1], token_features))
    df["csc_min_qd"]       = list(map(lambda x: x[2], token_features))
    df["csc_max_qd"]       = list(map(lambda x: x[3], token_features))
    df["ctc_min_qd"]       = list(map(lambda x: x[4], token_features))
    df["ctc_max_qd"]       = list(map(lambda x: x[5], token_features))
    df["last_word_eq_qd"]  = list(map(lambda x: x[6], token_features))
    df["first_word_eq_qd"] = list(map(lambda x: x[7], token_features))
    df["abs_len_diff_qd"]  = list(map(lambda x: x[8], token_features))
    df["mean_len_qd"]      = list(map(lambda x: x[9], token_features))
    # Merging Features with dataset - Token features for answer_text and distractor 
    token_features = df.apply(lambda x: get_token_features(x["answer_text"], x["distractor"]), axis=1)
    df["cwc_min_ad"]       = list(map(lambda x: x[0], token_features))
    df["cwc_max_ad"]       = list(map(lambda x: x[1], token_features))
    df["csc_min_ad"]       = list(map(lambda x: x[2], token_features))
    df["csc_max_ad"]       = list(map(lambda x: x[3], token_features))
    df["ctc_min_ad"]       = list(map(lambda x: x[4], token_features))
    df["ctc_max_ad"]       = list(map(lambda x: x[5], token_features))
    df["last_word_eq_ad"]  = list(map(lambda x: x[6], token_features))
    df["first_word_eq_ad"] = list(map(lambda x: x[7], token_features))
    df["abs_len_diff_ad"]  = list(map(lambda x: x[8], token_features))
    df["mean_len_ad"]      = list(map(lambda x: x[9], token_features))
    print("fuzzy features..")
    df["token_set_ratio_qd"]       = df.apply(lambda x: fuzz.token_set_ratio(x["question"], x["distractor"]), axis=1)
    df["token_set_ratio_ad"]       = df.apply(lambda x: fuzz.token_set_ratio(x["answer_text"], x["distractor"]), axis=1)
    df["token_sort_ratio_qd"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["question"], x["distractor"]), axis=1)
    df["token_sort_ratio_ad"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["answer_text"], x["distractor"]), axis=1)
    df["fuzz_ratio_qd"]            = df.apply(lambda x: fuzz.QRatio(x["question"], x["distractor"]), axis=1)
    df["fuzz_ratio_ad"]            = df.apply(lambda x: fuzz.QRatio(x["answer_text"], x["distractor"]), axis=1)
    df["fuzz_partial_ratio_qd"]    = df.apply(lambda x: fuzz.partial_ratio(x["question"], x["distractor"]), axis=1)
    df["fuzz_partial_ratio_ad"]    = df.apply(lambda x: fuzz.partial_ratio(x["answer_text"], x["distractor"]), axis=1)
    df["longest_substr_ratio_qd"]  = df.apply(lambda x: get_longest_substr_ratio(x["question"], x["distractor"]), axis=1)
    df["longest_substr_ratio_ad"]  = df.apply(lambda x: get_longest_substr_ratio(x["answer_text"], x["distractor"]), axis=1)
    print(df.shape)
    q_vec = pd.DataFrame()
    a_vec = pd.DataFrame()
    d_vec = pd.DataFrame()
    ques_vec = [get_word_vec(df.values[0][0], 'question')]
    ans_vec = [get_word_vec(df.values[0][1], 'option')]
    for i in df.values:
        q_vec = q_vec.append(ques_vec, ignore_index=True)
        a_vec = a_vec.append(ans_vec, ignore_index=True)
        d_vec = d_vec.append([X_opt_vec.get(i[2], np.zeros(100))], ignore_index=True)
    
    df = pd.concat([df, q_vec, a_vec, d_vec], axis = 1)
    print(df.shape)
    X_test = (df.drop(['question', 'answer_text', 'distractor'], axis=1)).to_numpy()
    df['target'] = model.predict(X_test)
    return df[df.target==1]


In [0]:
%%time
test_data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/ValueLabs/Results.csv")
print(pd.DataFrame(test_data.head(1)).columns)
result = predict_distractor(test_data.loc[[1]])

Index(['question', 'answer_text', 'distractor'], dtype='object')
['in the summer high season  finland does nt seem to sleep because'
 'the sun is out at night' 'makes us more clever']
fuzzy features..
(99388, 45)
(99388, 345)
CPU times: user 1h 31min 44s, sys: 2min, total: 1h 33min 44s
Wall time: 1h 33min 50s


In [0]:
print("Question")
print(result.iloc[0].question)
print("Answer")
print(result.iloc[0].answer_text)
print("Distractors")
result.iloc[200:205].distractor

Question
in the summer high season  finland does nt seem to sleep because
Answer
the sun is out at night
Distractors


1226    contributed to the invention of atomic bomb
1228     true happiness comes from spiritual riches
1230         the person who called the firefighters
1241      different people have different attitudes
1247                               the walk a thons
Name: distractor, dtype: object

# Extensions


In [0]:
def get_top_n_similiarities(vec1, vec2):
    n=3
    distances = []
    for index, i in enumerate(v2):
        vec2 = train_data_vec[index]
        d = np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
        if len(distances) < n:
            distances.append((i,d))
        else:
            distances.sort(key=lambda x: x[1], reverse=True)
            if (distances[-1][1]<d):
                distances.pop()
                distances.append((i,d))
    result = ''
    for index, i in enumerate(distances):
        sen = ' '.join(i[0])
        result = result + "'"+sen+"'"
        if index!=len(distances)-1:
            result+=","
    return result


#References
1. https://stackoverflow.com/questions/16096627/selecting-a-row-of-pandas-series-dataframe-by-integer-index